<a href="https://colab.research.google.com/github/sleter/devmeetings_datascience/blob/master/notebooks/DM_Data_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tworzenie własnego potoku transformującego dane

### Tworzenie niestandardowych transformatorów

Jak zauważyliśmy biblioteka Scikit-Learn posiada wiele przydatnych funkcji tranformujących jak np. SimpleImputer czy LabelEncoder. Czasami jednak jest potrzeba zdefiniowania własnej funkcji transformującej.

Dzięki mechanizmowi dziedziczenia w bibliotece Scikit-Learn, żeby dodać własny tranformator wystarczy stworzyć klasę i zaimportować 3 metody: *fit*, *transform* oraz *fit_transform*.

Metoda *fit_transform* jest połączeniem *fit* i *transform* i nie trzeba jej implementować przy dziedziczeniu po klasie TransformerMixer.

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

data = {
    'name':['Tom', 'Nick', 'Tom', 'Jack'],
    'age':[20, 21, 19, 18],
    'value':[3, 5, 2, 11]}
    
df_ = pd.DataFrame(data)

df_test = df_.copy()
df_test['new_value'] = df_['age']*df_['value']
print(df_test.head())

class NewTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, multiple_by_3 = True, age_index = 1, value_index = 2):
    self.multiple_by_3 = multiple_by_3
    self.age_index = age_index
    self.value_index = value_index
  
  def fit(self, X, y = None):
    return self
  
  def transform(self, X, y = None):
    new_value = X[:, self.age_index]*X[:, self.value_index]
    if self.multiple_by_3:
      new_value = new_value*3
    return np.c_[X, new_value]

nt = NewTransformer(multiple_by_3=False)
print(nt.fit_transform(df_.values))

nt = NewTransformer()
print(nt.fit_transform(df_.values))


   name  age  value  new_value
0   Tom   20      3         60
1  Nick   21      5        105
2   Tom   19      2         38
3  Jack   18     11        198
[['Tom' 20 3 60]
 ['Nick' 21 5 105]
 ['Tom' 19 2 38]
 ['Jack' 18 11 198]]
[['Tom' 20 3 180]
 ['Nick' 21 5 315]
 ['Tom' 19 2 114]
 ['Jack' 18 11 594]]


### Potoki transformujące

Poznaliśmy juz wiele różnych operacji na danych i transformatów.

Trzeba jednak pamiętać ciągle w jakiej kolejności je uruchamiać nie pomijając przy tym jakiejś operacji. Tutaj z pomocą przychodza potoki transformujące pozwalające ustalać kolejkę wykonywanych transformacji.

In [0]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__(self, attributes_names):
    self.attributes_names = attributes_names
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X[self.attributes_names].values

In [3]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

num_columns = list(df_.columns)
num_columns.remove("name")
cat_columns = ["name"]

cat_pipeline = Pipeline([
  ('df_selector', DataFrameSelector(cat_columns)),
  ('label_encoder', OneHotEncoder(sparse=False)),
])

num_pipeline = Pipeline([
  ('df_selector', DataFrameSelector(num_columns)),
  ('imputer', SimpleImputer()),
  ('new_transformer', NewTransformer(age_index = 0, value_index = 1)),
  ('standard_scaler', StandardScaler())
])

full_pipeline = FeatureUnion(transformer_list=[
  ('cat_pipeline', cat_pipeline),
  ('num_pipeline', num_pipeline)
])

print("Before: ")
print(df_)
final_df_ = full_pipeline.fit_transform(df_)
print("\nAfter: ")
print(final_df_)

Before: 
   name  age  value
0   Tom   20      3
1  Nick   21      5
2   Tom   19      2
3  Jack   18     11

After: 
[[ 0.          0.          1.          0.4472136  -0.64450339 -0.65569162]
 [ 0.          1.          0.          1.34164079 -0.07161149  0.07737976]
 [ 0.          0.          1.         -0.4472136  -0.93094934 -1.01408207]
 [ 1.          0.          0.         -1.34164079  1.64706421  1.59239393]]


### Zadania



1.   Stwórz własny transformator dla wybranych transformacji dla zbioru danych housing.csv napisanych w notebooku 2 w punkcie "Zadania" w podpunkcie 5.
2.   Stwórz własny potok transformujący dla zbioru danych housing.csv zawierający osobne operacje dla kolumn kategorycznych i numerycznych.
* **Dla kategorycznych** mogą to być operacje wybrania odpowiedniego podzbioru atrybutów kategorycznych i zastąpienia ich macierzą rzadką.
* **Dla numerycznych** mogą to być operacje wybrania odpowiedniego podzbioru atrybutów numerycznych, zastąpienie nieznanych wartości średnią, użycie transformatora z zadania wyżej i przeskalowanie wszystkich wartości.

In [0]:
# WRITE YOUR CODE HERE